In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

60480

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 2520)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2400)

In [15]:
x[:,:2]

matrix([[  2512.07934066,   2661.06304182],
        [  3369.19698782,   3121.9936462 ],
        [  3924.76103391,   3445.18498449],
        [  5634.67717605,   4665.40401906],
        [  4263.86165094,   4327.47518773],
        [  5480.27236726,   5330.31756928],
        [  4405.85100893,   3565.33620988],
        [  3361.22468207,   2470.16402502],
        [  5430.75753853,   8578.34929294],
        [ 14473.65164558,  12400.00694082],
        [  5888.79539117,   7076.58624941],
        [ 10729.44161845,  10732.86354829],
        [  8488.22127328,  12505.48508171],
        [  3092.40157496,   2329.3003975 ],
        [  8943.05790073,   8437.26368043],
        [  1135.40582207,    996.78831378],
        [  3527.28707193,   3418.49361055],
        [  3171.08535536,   2240.91484119],
        [ 13547.04557644,  12186.94832528],
        [  4265.65421257,   3511.62380378],
        [ 12264.01747983,  12295.70442627],
        [  5068.39133399,   5084.66882832],
        [  3919.81375018,   3906

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e-02, 1e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.83256895e+06  0.00000000e+00  4.92e+03 1.98e+02  9.39e+05     0s
   1   9.54741926e+05 -1.90694228e+06  7.63e+02 1.46e+02  1.92e+05     0s
   2   4.79284784e+05 -1.02551522e+06  7.63e-04 1.14e-13  2.69e+04     0s
   3  -6.49051875e+04 -2.75215743e+05  7.64e-10 5.68e-14  3.7

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.3473679361259703e-10,
 2807.5279406675036,
 1.4789527570941162e-10,
 6.664381284871544e-07,
 1340.185698734798,
 1.117331191008154e-06,
 4.292180266998288e-08,
 0.0008002217637364528,
 883.4208140992027,
 3190.989735905362,
 6.66438366760504e-07,
 581.764308809899,
 1.1173318479776957e-06,
 1.1237312171175182e-06,
 5901.698371039733,
 6786.292240388291,
 6.664382737235706e-07,
 6.664380364676905e-07,
 1841.6143193380456,
 9939.582535475809,
 1.1170939176753907e-06,
 2668.0948797012643,
 1092.223879663977,
 1249.1429758493898,
 2591.0499451079118,
 1929.568514399647,
 1786.1854149768706,
 3397.0359697039776,
 0.05265260077882594,
 0.016221712204705252,
 4452.8343550655845,
 1299.6803823694868,
 4603.180819429563,
 2580.544048707163,
 1031.5373153905373,
 5.1912698171565144e-05,
 5.1912654500474095e-05,
 6241.137400042277,
 688.7632924300725,
 1729.9460846349782,
 5002.914410416363,
 1140.9183564847178,
 6.650884976128917e-07,
 6.650884885248923e-07,
 6377.683168294559,
 3.31551418306